# 처음 문서들을 임베딩하여 DB에 저장하는 파일
- 처음 한 번만 실행하면 됨

# 1. Loader

## DirectoryLoader + PDFPlumberLoader
- 범용성과 가장 성능이 좋은 로더 결합

### DirectoryLoader
상대경로? 절대경로?
- .하나 : 현재경로 -> ./(동일폴더)
- .. : 상위경로 -> ../(상위폴더)

### PDFPlumberLoader
- 가장 깔끔하게 pdf내용들을 가져옴
- 날짜 표시
- 전체 페이지 표시

In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PDFPlumberLoader


# 디렉토리로더 초기화
loader = DirectoryLoader("./data", glob="**/*.pdf",loader_cls=PDFPlumberLoader, show_progress=True)
    # ./는 현재 디렉토리를 의미하고, 그 하위에 있는 'data' 폴더
    # glob는 파일을 검색할 때 사용할 패턴   
    # **/는 모든 하위 디렉토리
    # *.pdf는 확장자가 .pdf인 모든 파일

# 문서 로드
docs = loader.load()

# 문서 개수 계산
len(docs)

100%|██████████| 10/10 [00:16<00:00,  1.61s/it]


317

# 2. Splitter

## RecursiveCharacterTextSplitter

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

split_docs = text_splitter.split_documents(docs)

print(len(split_docs))

2139


# 3. Embedding

In [3]:
# 단계 3: 임베딩(Embedding) 생성
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device":"cpu"}
encode_kwargs = {"normalize_embeddings":True}

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs,encode_kwargs=encode_kwargs)

/home/jun/.conda/envs/chromagroq/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# 4. VectorDB

## Chroma

In [4]:
from langchain_chroma import Chroma


db = Chroma.from_documents(documents=split_docs, embedding=embeddings,
                           collection_name="Low_chroma_300chunk_db", persist_directory="./low")
# collection_name : Low_chroma_db , Low_chroma_300chunk_db
# collection_name : 폴더의 개념
# persist_directory : 이 경로로 저장 -> 차후 다시 사용가능

# 5. Retriever
잘 됐는지 확인

In [5]:
ans = db.similarity_search("보증금 회수 어떻게 해?",k=2) 

In [6]:
print(ans[0].page_content)

공하거나 보증인을 면책하게 함으로써 그 배상의무를 면할 수 있다.
제444조(부탁없는 보증인의 구상권) ①주채무자의 부탁없이 보증인이 된 자가 변제 기타 자기의 출재로 주채무를 소멸
하게 한 때에는 주채무자는 그 당시에 이익을 받은 한도에서 배상하여야 한다.
②주채무자의 의사에 반하여 보증인이 된 자가 변제 기타 자기의 출재로 주채무를 소멸하게 한 때에는 주채무자는
현존이익의 한도에서 배상하여야 한다.
③전항의 경우에 주채무자가 구상한 날 이전에 상계원인이 있음을 주장한 때에는 그 상계로 소멸할 채권은 보증인
에게 이전된다.


In [7]:
print(ans[1].page_content)

으로 본다.<개정 2013. 8. 13.>
⑤ 이 법에 따라 임대차의 목적이 된 주택이 매매나 경매의 목적물이 된 경우에는 「민법」 제575조제1항ㆍ제3항 및
같은 법 제578조를 준용한다.<개정 2013. 8. 13.>
⑥ 제5항의 경우에는 동시이행의 항변권(抗辯權)에 관한 「민법」 제536조를 준용한다.<개정 2013. 8. 13.>
[전문개정 2008. 3. 21.]
제3조의2(보증금의 회수) ① 임차인(제3조제2항 및 제3항의 법인을 포함한다. 이하 같다)이 임차주택에 대하여 보증금


#### Ref
- [법률데이터(pdf)](https://www.easylaw.go.kr/CSP/SysChartRetrievePLst.laf?csmSeq=629) <- 생활법률